In [1]:
import numpy                      as np
import tensorflow                 as tf
import tensorflow_hub             as hub
import tensorflow_datasets        as tfds

# Data

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    'mnist',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

## Pre-porcess image

In [3]:
strategy = tf.distribute.MirroredStrategy()

print('number of devices {}'.format(strategy.num_replicas_in_sync))

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:CPU:0',)


number of devices 1


In [4]:
## batch size based on number of devices

BATCH_SIZE_PER_REPLICA = 64
# Use for Mirrored Strategy
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [5]:
def normalize(img, label):
  return tf.cast(img, float)/255., label

In [6]:
ds_train = ds_train.map(normalize, num_parallel_calls= tf.data.experimental.AUTOTUNE)
ds_train = ds_train.shuffle(buffer_size=1000).batch(BATCH_SIZE)
ds_train = ds_train.prefetch(1000)

In [7]:
ds_test = ds_test.map(normalize, num_parallel_calls= tf.data.experimental.AUTOTUNE)
ds_test = ds_test.shuffle(buffer_size=1000).batch(BATCH_SIZE)
ds_test = ds_test.prefetch(1000)

## Model

In [8]:
with strategy.scope():
  model = tf.keras.Sequential([
                            tf.keras.layers.Conv2D(64, 3, activation='relu', input_shape = (28,28,1)),
                            tf.keras.layers.MaxPool2D(2),
                            tf.keras.layers.Flatten(),
                            tf.keras.layers.Dense(64, activation='relu'),
                            tf.keras.layers.Dense(10, activation='softmax')
  ])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
flatten (Flatten)            (None, 10816)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                692288    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 693,578
Trainable params: 693,578
Non-trainable params: 0
_________________________________________________________________


## Optimizer, loss and accuracy

In [9]:
model.compile(optimizer='adam', loss=tf.losses.sparse_categorical_crossentropy,
              metrics = ['acc'])

model.fit(ds_train, epochs=2)

Epoch 1/2
938/938 [==============================] - 42s 42ms/step - loss: 0.3440 - acc: 0.9029
Epoch 2/2
938/938 [==============================] - 36s 38ms/step - loss: 0.0642 - acc: 0.9807
